# Self-Driving Car Engineer Nanodegree

## Deep Learning

## Project: Build a Traffic Sign Recognition Classifier

In this notebook, a template is provided for you to implement your functionality in stages which is required to successfully complete this project. If additional code is required that cannot be included in the notebook, be sure that the Python code is successfully imported and included in your submission, if necessary. Sections that begin with **'Implementation'** in the header indicate where you should begin your implementation for your project. Note that some sections of implementation are optional, and will be marked with **'Optional'** in the header.

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

---

## Step 1: Dataset Exploration

Visualize the German Traffic Signs Dataset. This is open ended, some suggestions include: plotting traffic signs images, plotting the count of each sign, etc. Be creative!


The pickled data is a dictionary with 4 key/value pairs:

- features -> the images pixel values, (width, height, channels)
- labels -> the label of the traffic sign
- sizes -> the original width and height of the image, (width, height)
- coords -> coordinates of a bounding box around the sign in the image, (x1, y1, x2, y2). Based the original image (not the resized version).

In [117]:
import pickle
import pandas as pd
import math
import os
import numpy as np
from itertools import product
import cv2
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
import tensorflow as tf
from tensorflow.python.ops.variables import Variable
from tensorflow.contrib.layers import flatten
import helpers as h

In [118]:
# TODO: fill this in based on where you saved the training and testing data
training_file = 'data/train.p'
testing_file = 'data/test.p'

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
    
X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

import pandas as pd
signnames = pd.read_csv('data/signnames.csv')

In [119]:
### To start off let's do a basic data summary.
n_train = len(X_train)
n_test = len(X_test)
image_shape = X_train.shape[1]
n_classes = len(set(y_train))

print("Number of training examples =", n_train)
print("Number of testing examples =", n_test)
print("Image data shape =", image_shape)
print("Number of classes =", n_classes)

Number of training examples = 39209
Number of testing examples = 12630
Image data shape = 32
Number of classes = 43


Let's take a quick look at the distribution of the classes:

In [120]:
from bokeh.charts import Histogram
from bokeh.charts import defaults, vplot, hplot, show, output_notebook
defaults.plot_width = 900
defaults.plot_height = 300
output_notebook(hide_banner=True)
hist = Histogram(y_train, bins=42) #, x_axis_label='Documents', y_axis_label='Sparsity')
hist.xaxis.axis_label = 'Class'
hist.yaxis.axis_label = 'Frequency'
show(hist)

In [121]:
%matplotlib inline

print("Training data shape = ", X_train.shape)
print("Number of channels in each image = ", X_train.shape[3])

if ( not os.path.exists('signs.png')):
    idx = sample_indices(X_train, y_train, 10)
    fig = plot_images(X_train[idx,:,:,:], 10)
    fig.savefig('signs.png',bbox_inches="tight" )    

Training data shape =  (39209, 32, 32, 3)
Number of channels in each image =  3


Here are a sample of each sign:

<img src="signs.png" alt="signs.png" width="400">

There are some features of the data that we will need to address:

* Class imbalance
* Poor quality images - due to:
    * motion blur
    * low light conditions
    * low contrast images
    * high exposure images
    * obstruction due to shadows
    * obstruction due to graffiti

Here are some observations that may inform choice of data preprocessing and machine learning approach:

* All images contain traffic signs - so we are only concerned with classification not detection
* The signs are largely centred in the image
* The data already contains transformed (jittered) versions of images which will help when training deep convolutional networks.


----

## Step 2: Design and Test a Model Architecture

Design and implement a deep learning model that learns to recognize traffic signs. Train and test your model on the [German Traffic Sign Dataset](http://benchmark.ini.rub.de/?section=gtsrb&subsection=dataset).

There are various aspects to consider when thinking about this problem:

- Your model can be derived from a deep feedforward net or a deep convolutional network.
- Play around preprocessing techniques (normalization, rgb to grayscale, etc)
- Number of examples per label (some have more than others).
- Generate fake data.

Here is an example of a [published baseline model on this problem](http://yann.lecun.com/exdb/publis/pdf/sermanet-ijcnn-11.pdf). It's not required to be familiar with the approach used in the paper but, it's good practice to try to read papers like these.

### Implementation

Use the code cell (or multiple code cells, if necessary) to implement the first step of your project. Once you have completed your implementation and are satisfied with the results, be sure to thoroughly answer the questions that follow.

### Data Preprocessing

The images were preprocessed in the follwing manner:
* Convert orginal RGB images to grayscale
* Apply histogram equalization to each image

For this project I only considered grayscale images.  I did this to reduce the dimensionality of the data - however I am aware that by doing this I was discarding potentially useful information.

I also applied histogram equalization in order to maximise the distribution of pixel values in each image.  Here is the preprocessing I performed:

In [122]:
X_train_gray = np.empty( [n_train, image_shape, image_shape], dtype = np.int32)
for i, img in enumerate(X_train):
    X_train_gray[i,:,:] = cv2.equalizeHist(h.grayscale(img), (0, 254) )
X_test_gray = np.empty( [n_test, image_shape, image_shape], dtype = np.int32)
for i, img in enumerate(X_test):
    X_test_gray[i,:,:] = cv2.equalizeHist(h.grayscale(img), (0, 254) )
    
if ( not os.path.exists('signs-transformed.png')):
    idx = sample_indices(X_train, y_train, 10)
    fig = plot_images(X_train_gray[idx,:,:], 10, grayscale=True)
    fig.savefig('signs-transformed.png',bbox_inches="tight" )  

Note that I placed supporting functions in the `helpers.py` module - imported as `h`.  You can see my `helpers.py` [here](helpers.py).  Creating this library of functiosn made it easier to run my code in notebooks or standalone when training on AWS.

Here are a sample of the transformed images:

<img src="signs-transformed.png" alt="signs-transformed.png" width="400">

### Training / Validation Data

I split the training data into two groups - one for training and one for validation.  We do not touch the test data set.  That is reserved until the end to estimate overall performance.  I reserved 10% of the original training data for validation - the reamining 90% formed the training data.

Here is the code to split the data set:

In [123]:
X_train_gray, X_valid_gray, y_train, y_valid = train_test_split( 
    X_train_gray,
    y_train,
    test_size=0.10,
    random_state=1973)

### Label Encoding

I encoded the target variable using one-hot encoding:

In [124]:
encoder = LabelBinarizer()
encoder.fit(y_train)
train_labels = encoder.transform(y_train)
valid_labels = encoder.transform(y_valid)
test_labels = encoder.transform(y_test)

### Reshaping and Normalization

Finally in preparation to use Tensorflow I converted the labels to be of type `np.float32`.  I also flattened hte images into single vectors of length 32 x 32 = 1024. Finally I normalized all the values in the training data to lie in the range 0.1 to 0.9. 

In [125]:
# Change to float32 for Tensorflow
train_labels = train_labels.astype(np.float32)
valid_labels = valid_labels.astype(np.float32)
test_labels = test_labels.astype(np.float32)

X_train_gray_flat = h.flatten_all_gray(X_train_gray)
X_valid_gray_flat = h.flatten_all_gray(X_valid_gray)
X_test_gray_flat = h.flatten_all_gray(X_test_gray)

X_train_gray_flat = h.normalize_grayscale(X_train_gray_flat)
X_valid_gray_flat = h.normalize_grayscale(X_valid_gray_flat)
X_test_gray_flat = h.normalize_grayscale(X_test_gray_flat)

### Model Architecture

Since this was the first time using Tensorflow I wanted to approach things as simply and incrementally as possible!  My plans for model architecture were to proceed as follows:

* Start with simple linear classifier on the data directly - like the previous lab - to test my use of tensorflow
* Then make deeper traditional multi-layer (fully connected) neural networks
* Finally build a convnet - we will choose LeNet since we studied that in the lab

To train all models I used stochastic gradient descent (minibatch).   Output layers to all networks applied softmax to logits to get probability distribution across the multiple classes.   I used the cross entropy error function.  Between hidden layers I include relus.  Weights will be initialised to small random values sampled from a truncated_normal distribution.

Before I started I created functions to implement Early Stopping, to serialize model and performance data.  I reused the same code for all the different model architectures.  These functions can be seen in the `helper.py` module [here](helpers.py).

Here's the story:

Worked in grayscale.

Used AWS - so wrote standalone python code.  Link to programs.  

I also implemented early stopping
Trained a simple linear classifer - show train-lin.py.  Load model and show performance.
Trained a fully connected.  WOrked better
Trained a cnn - worked best

I was also interested in the effect of rebalancing the data - so I jittered the images to create a dataset that had the same number of images per class.  

Load the checkpoints and score the validation data - visualise some of the prediction errors and contingency matrices.

Evaulate the best model on the test data

Next steps:
Color images?
Explore why resampling not helping?  Maybe too crude - and introducing dark patches that may be being picked up by the classifier.





### Linear Classifier

Lets start with linear classifier - single layer implementing a simple Linear Function WX + b:

In [ ]:
import tensorflow as tf
from tensorflow.python.ops.variables import Variable

features_count = 32 * 32

features = tf.placeholder(tf.float32, [None, features_count])
labels = tf.placeholder(tf.float32, [None, n_classes]) 

weights = tf.Variable(tf.truncated_normal(stddev=0.1, dtype=tf.float32, shape=[features_count, n_classes]))
biases = tf.Variable(tf.zeros([n_classes]))

train_feed_dict = {features: X_train_gray_flat, labels: train_labels}
valid_feed_dict = {features: X_valid_gray_flat, labels: valid_labels}
test_feed_dict = {features: X_test_gray_flat, labels: test_labels}

logits = tf.matmul(features, weights) + biases

prediction = tf.nn.softmax(logits)
predicted_class = tf.argmax(prediction, dimension=1)

cross_entropy = -tf.reduce_sum(labels * tf.log(prediction), reduction_indices=1)
loss = tf.reduce_mean(cross_entropy)

init = tf.initialize_all_variables()

with tf.Session() as session:
    session.run(init)
    session.run(loss, feed_dict=train_feed_dict)
    session.run(loss, feed_dict=valid_feed_dict)
    session.run(loss, feed_dict=test_feed_dict)
    biases_data = session.run(biases)
    
is_correct_prediction = tf.equal(tf.argmax(prediction, 1), tf.argmax(labels, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct_prediction, tf.float32))

In [ ]:
train_linear = False
if train_linear:
    result = train_model(model_name='linear',
                            init=init,
                            loss=loss,
                            features=features,
                            labels=labels,
                            X_train= X_train_gray_flat,
                            y_train=train_labels,
                            train_feed_dict=train_feed_dict,
                            valid_feed_dict=valid_feed_dict,
                            accuracy=accuracy,
                            predicted_class=predicted_class,
                            batch_size=100, 
                            epochs=2000,
                            learning_rate=0.2,
                            early_stopping_rounds = 10)
    serialize_training_data('models/linear-train', result)
    train_linear = False
else:
    result = deserialize_training_data('models/linear-train')

training_plots(result)
print('Validation accuracy at {}'.format(result['validation_accuracy']))
cm = confusion_matrix(y_true=y_valid,
                          y_pred=result['y_pred'])
output_notebook(hide_banner=True)
show(plot_contingency_matrix(cm, n_classes, list(signnames['SignName'])))

Surprisingly good accuracy for such a simple network.  Let's add a hidden layer.

### Single Hidden Layer

Here we implement a single hidden layer of width 200.  Also I updated the code to include **early stopping**.  Easier to let the network train for many iterations - stopping if no improvement after a specified number of iterations.

In [ ]:
import tensorflow as tf
from tensorflow.python.ops.variables import Variable

features_count = 32 * 32
n_hidden_layer = 200

features = tf.placeholder(tf.float32, [None, features_count])
labels = tf.placeholder(tf.float32, [None, n_classes]) 

weights = {
    'hidden_layer': tf.Variable(tf.random_normal(shape=[features_count, n_hidden_layer], stddev=0.05, dtype=tf.float32)),
    'output_layer': tf.Variable(tf.random_normal(shape=[n_hidden_layer, n_classes], stddev=0.05, dtype=tf.float32))
}
biases = {
    'hidden_layer': tf.Variable(tf.random_normal(shape=[n_hidden_layer], stddev=0.05, dtype=tf.float32)),
    'output_layer': tf.Variable(tf.random_normal(shape=[n_classes], stddev=0.05, dtype=tf.float32))
}

out = tf.add(tf.matmul(features, weights['hidden_layer']), biases['hidden_layer'])
out = tf.nn.relu(out)
logits = tf.matmul(out, weights['output_layer']) + biases['output_layer']
prediction = tf.nn.softmax(logits)
predicted_class = tf.argmax(prediction, dimension=1)

train_feed_dict = {features: X_train_gray_flat, labels: train_labels}
valid_feed_dict = {features: X_valid_gray_flat, labels: valid_labels}
test_feed_dict = {features: X_test_gray_flat, labels: test_labels}

cross_entropy = -tf.reduce_sum(labels * tf.log(prediction), reduction_indices=1)
loss = tf.reduce_mean(cross_entropy)

init = tf.initialize_all_variables()

with tf.Session() as session:
    session.run(init)
    session.run(loss, feed_dict=train_feed_dict)
    session.run(loss, feed_dict=valid_feed_dict)
    session.run(loss, feed_dict=test_feed_dict)
    biases_data = session.run(biases)
    
is_correct_prediction = tf.equal(tf.argmax(prediction, 1), tf.argmax(labels, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct_prediction, tf.float32))

In [ ]:
train_fully_connected=False
if train_fully_connected:
    result = train_model(model_name='fully_connected',
                            init=init,
                            loss=loss,
                            features=features,
                            labels=labels,
                            X_train= X_train_gray_flat,
                            y_train=train_labels,
                            train_feed_dict=train_feed_dict,
                            valid_feed_dict=valid_feed_dict,
                            accuracy=accuracy,
                            predicted_class=predicted_class,batch_size=100, 
                            epochs=2000,
                            learning_rate=0.05,
                            early_stopping_rounds = 10)
    serialize_training_data('models/fully_connected', result)
    train_linear = False
else:
    result = deserialize_training_data('models/fully_connected')

training_plots(result)
print('Validation accuracy at {}'.format(result['validation_accuracy']))
cm = confusion_matrix(y_true=y_valid,
                          y_pred=result['y_pred'])
output_notebook(hide_banner=True)
show(plot_contingency_matrix(cm, n_classes, list(signnames['SignName'])))

Which images are not being classified correctly?  Here we display images that are misclassified into a class - the first two images represent correct classifications, followed by upto 8 images that we incorrectly classified as the same class:

In [ ]:
idx = sample_misclassified(result['y_pred'], y_valid)
plot_image_grid(X_valid_gray, idx, grayscale = True)

We could train a deeper network - but I think it may be better to build a convolutional neural network.  I think we might get similar performance but quicker.

### Convnet

I am going to start with LeNet from the lecture.

First let's pull in helper functions to quickly define layers:

In [ ]:
def LeNet(x):
    x = tf.reshape(x, (-1, 32, 32, 1))
    # Convolution layer 1. The output shape should be 28x28x6.
    x=conv_layer(input=x, num_input_channels=1, filter_size=5, num_filters=6, stride=1, padding='VALID')
    x=tf.nn.relu(x)
    # Pooling layer 1. The output shape should be 14x14x6.
    x=tf.nn.max_pool(value=x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    # Convolution layer 2. The output shape should be 10x10x16.
    x=conv_layer(input=x, num_input_channels=6, filter_size=5, num_filters=16, stride=1, padding='VALID')
    x=tf.nn.relu(x)
    # Pooling layer 2. The output shape should be 5x5x16.
    x=tf.nn.max_pool(value=x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    # Flatten layer. Flatten the output shape of the final pooling layer such that it's 1D instead of 3D. The easiest way to do is by using tf.contrib.layers.flatten, which is already imported for you.
    x=tf.contrib.layers.flatten(x)
    # Fully connected layer 1. This should have 120 outputs.
    x=fully_connected_layer(x, 400, 120)
    x=tf.nn.relu(x)
    # Fully connected layer 2. This should have 43 outputs.
    x=fully_connected_layer(x, 120, 43)
    # Return the result of the last fully connected layer.
    return x

In [ ]:
import tensorflow as tf
from tensorflow.python.ops.variables import Variable

features_count = 32 * 32

features = tf.placeholder(tf.float32, [None, features_count])
labels = tf.placeholder(tf.float32, [None, n_classes]) 

# Note we reshape flattened feature back to 32x32 in LeNet:
leNet = LeNet(features)

prediction = tf.nn.softmax(leNet)
predicted_class = tf.argmax(prediction, dimension=1)

train_feed_dict = {features: X_train_gray_flat, labels: train_labels}
valid_feed_dict = {features: X_valid_gray_flat, labels: valid_labels}
test_feed_dict = {features: X_test_gray_flat, labels: test_labels}

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(leNet, labels))

init = tf.initialize_all_variables()

with tf.Session() as session:
    session.run(init)
    session.run(loss, feed_dict=train_feed_dict)
    session.run(loss, feed_dict=valid_feed_dict)
    session.run(loss, feed_dict=test_feed_dict)
    #biases_data = session.run(biases)
    
is_correct_prediction = tf.equal(tf.argmax(prediction, 1), tf.argmax(labels, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct_prediction, tf.float32))

In [110]:
%run helpers.py

train_lenet=False
if train_lenet:
    result = train_model(model_name='lenet',
                            init=init,
                            loss=loss,
                            features=features,
                            labels=labels,
                            X_train= X_train_gray_flat,
                            y_train=train_labels,
                            train_feed_dict=train_feed_dict,
                            valid_feed_dict=valid_feed_dict,
                            accuracy=accuracy,
                            predicted_class=predicted_class,
                            batch_size=100, 
                            epochs=2,
                            learning_rate=0.001,
                            early_stopping_rounds = 10,
                            opt="GD")
    serialize_training_data('models/lenet', result)
    train_linear = False
else:
    result = deserialize_training_data('models/lenet')

training_plots(result)
print('Validation accuracy at {}'.format(result['validation_accuracy']))
cm = confusion_matrix(y_true=y_valid,
                          y_pred=result['y_pred'])
output_notebook(hide_banner=True)
show(plot_contingency_matrix(cm, n_classes, list(signnames['SignName'])))

Validation accuracy at 0.8138229846954346


You can see that the CNN is not performing well.  I ran a few experiments to see where the problem might be happening.  At first I thought the CNN size may have been too small - so I increased the size of the network - introducing an additional convolutional layer and an additional fully connected layer.  My architecture was now:

[insert diagram]

This helped but still my validation accuracy was not getting above 0.86.

I decided that maybe creation of fake (augmented) data would help.  



### Image Augmentation

There is class imbalance in the training data - I fixed this imbalance by creating additional images from the training data.  In particular I applied a random X and Y translation and a random rotation.  I used python generators to return balanced samples across all the classes. 

In [109]:
def bin_data(X, y):
    Xb={}
    for i in range(0,X.shape[0]):
        bin = y[i]
        if bin in Xb:
            Xb[bin].append(X[i,:,:])
        else:
            Xb[bin] = [X[i,:,:]]
    return Xb

def transform_image(img):
    
    X_OFFSET_RANGE = 10
    Y_OFFSET_RANGE = 10
    ROTATION_RANGE = 50
    
    rows,cols = img.shape 
    
    img = np.float32(img)
    
    # Random rotation
    angle = np.random.uniform(ROTATION_RANGE)-ROTATION_RANGE/2
    t_rot = cv2.getRotationMatrix2D((cols/2,rows/2),angle,1)
    
    # Random translation
    x_translation = (X_OFFSET_RANGE * np.random.uniform()) - (X_OFFSET_RANGE / 2)
    y_translation = (Y_OFFSET_RANGE * np.random.uniform()) - (Y_OFFSET_RANGE / 2)
    t_trans = np.float32([[1, 0, x_translation], [0, 1, y_translation]])
    
    img = cv2.warpAffine(img, t_trans, (cols, rows))
    img = cv2.warpAffine(img, t_rot, (cols, rows))

    img = np.int32(img)
    
    return(img)
    
def image_generator(Xb, n):
    X=[]
    y=[]
    n_classes = len(Xb)
    for i in range(0,n):
        idb = np.random.randint(0,n_classes)
        n_images = len(Xb[idb])
        idx = np.random.randint(0,n_images)
        X.append(transform_image(Xb[idb][idx]))
        y.append(idb)
    yield (X, y)


X_train_gray_b = bin_data(X_train_gray, y_train)

if ( not os.path.exists('signs-augmented.png')):
    g = image_generator(X_train_gray_b,100)
    fig = h.plot_images(g.__next__()[0], 10, grayscale=True)
    fig.savefig('signs-augmented.png') 

Here are a sample of augmented signs:

<img src="signs-augmented.png" alt="signs-augmented.png" width="400">

The generator also samples uniformly from all the sign classes:

In [108]:
from bokeh.charts import Bar
from bokeh.charts import defaults, vplot, hplot, show, output_notebook
defaults.plot_width = 900
defaults.plot_height = 300
g = image_generator(X_train_gray_b,10000)
d = np.histogram(g.__next__()[1], bins=range(0,44))
df = pd.DataFrame({'bin':d[1][:-1], 'freq':d[0]})
df
p = Bar(df, 'bin' , values='freq', legend=False)
p.yaxis.axis_label = 'Frequency'
show(p)

In [126]:
def LeNet2(x):
    x = tf.reshape(x, (-1, 32, 32, 1))
    # Convolution layer 1. The output shape should be 28x28x16.
    x=conv_layer(input=x, num_input_channels=1, filter_size=5, num_filters=16, stride=1, padding='VALID')
    x=tf.nn.relu(x)
    # Pooling layer 1. The output shape should be 14x14x16.
    x=tf.nn.max_pool(value=x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    # Convolution layer 2. The output shape should be 10x10x32.
    x=conv_layer(input=x, num_input_channels=16, filter_size=5, num_filters=32, stride=1, padding='VALID')
    x=tf.nn.relu(x)
    # Pooling layer 2. The output shape should be 5x5x32.
    x=tf.nn.max_pool(value=x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    # Flatten layer. Should have 5x5x32 = 800 outputs.
    x=tf.contrib.layers.flatten(x)
    # Fully connected layer 1. This should have 400 outputs.
    x=fully_connected_layer(x, 800, 400)
    x=tf.nn.relu(x)
    # Fully connected layer 2. This should have 100 outputs.
    x=fully_connected_layer(x, 400, 100)
    x=tf.nn.relu(x)
    # Fully connected layer 3. This should have 43 outputs.
    x=fully_connected_layer(x, 100, 43)
    # Return the result of the last fully connected layer.
    return x

In [161]:
%run helpers.py
import tensorflow as tf
from tensorflow.python.ops.variables import Variable

# Important in Notebooks!
tf.reset_default_graph()

features_count = 32 * 32

features = tf.placeholder(tf.float32, [None, features_count])
labels = tf.placeholder(tf.float32, [None, n_classes]) 

# Note we reshape flattened feature back to 32x32 in LeNet:
leNet = LeNet2(features)

prediction = tf.nn.softmax(leNet)
predicted_class = tf.argmax(prediction, dimension=1)

#train_feed_dict = {features: X_train_gray_flat, labels: train_labels}
valid_feed_dict = {features: X_valid_gray_flat, labels: valid_labels}
#test_feed_dict = {features: X_test_gray_flat, labels: test_labels}

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(leNet, labels))

init = tf.initialize_all_variables()

with tf.Session() as session:
    session.run(init)
    #session.run(loss, feed_dict=train_feed_dict)
    session.run(loss, feed_dict=valid_feed_dict)
    #session.run(loss, feed_dict=test_feed_dict)
    #biases_data = session.run(biases)
    
is_correct_prediction = tf.equal(tf.argmax(prediction, 1), tf.argmax(labels, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct_prediction, tf.float32))

X_train_gray_b = bin_data(X_train_gray, y_train)
g=data_generator(X_train_gray_b,10)


In [162]:
%run helpers.py

train_lenet=True
if train_lenet:
    result = train_model_generator(model_name='lenet2',
                            init=init,
                            loss=loss,
                            features=features,
                            labels=labels,
                            generator=g,
                            batches_per_epoch = 1,
                            valid_feed_dict=valid_feed_dict,
                            accuracy=accuracy,
                            predicted_class=predicted_class,
                            epochs=10,
                            learning_rate=0.001,
                            early_stopping_rounds = 1,
                            opt="Adam")
    serialize_training_data('models/lenet2', result)
    train_linear = False
else:
    result = deserialize_training_data('models/lenet2')

training_plots(result)
print('Validation accuracy at {}'.format(result['validation_accuracy']))
cm = confusion_matrix(y_true=y_valid,
                          y_pred=result['y_pred'])
output_notebook(hide_banner=True)
show(plot_contingency_matrix(cm, n_classes, list(signnames['SignName'])))

  0%|          | 0/10 [00:00<?, ?epochs/s]


FailedPreconditionError: Attempting to use uninitialized value beta2_power
	 [[Node: beta2_power/read = Identity[T=DT_FLOAT, _class=["loc:@Variable"], _device="/job:localhost/replica:0/task:0/cpu:0"](beta2_power)]]

Caused by op 'beta2_power/read', defined at:
  File "/home/mattwg/anaconda3/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/mattwg/anaconda3/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/mattwg/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/mattwg/anaconda3/lib/python3.5/site-packages/traitlets/config/application.py", line 653, in launch_instance
    app.start()
  File "/home/mattwg/anaconda3/lib/python3.5/site-packages/ipykernel/kernelapp.py", line 474, in start
    ioloop.IOLoop.instance().start()
  File "/home/mattwg/anaconda3/lib/python3.5/site-packages/zmq/eventloop/ioloop.py", line 162, in start
   

### Question 1 

_Describe the techniques used to preprocess the data._

**Answer:**

In [ ]:
### Generate data additional (if you want to!)
### and split the data into training/validation/testing sets here.
### Feel free to use as many code cells as needed.

### Question 2

_Describe how you set up the training, validation and testing data for your model. If you generated additional data, why?_

**Answer:**

In [ ]:
### Define your architecture here.
### Feel free to use as many code cells as needed.

### Question 3

_What does your final architecture look like? (Type of model, layers, sizes, connectivity, etc.)  For reference on how to build a deep neural network using TensorFlow, see [Deep Neural Network in TensorFlow
](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/b516a270-8600-4f93-a0a3-20dfeabe5da6/concepts/83a3a2a2-a9bd-4b7b-95b0-eb924ab14432) from the classroom._


**Answer:**

In [ ]:
### Train your model here.
### Feel free to use as many code cells as needed.

### Question 4

_How did you train your model? (Type of optimizer, batch size, epochs, hyperparameters, etc.)_


**Answer:**

### Question 5


_What approach did you take in coming up with a solution to this problem?_

**Answer:**

---

## Step 3: Test a Model on New Images

Take several pictures of traffic signs that you find on the web or around you (at least five), and run them through your classifier on your computer to produce example results. The classifier might not recognize some local signs but it could prove interesting nonetheless.

You may find `signnames.csv` useful as it contains mappings from the class id (integer) to the actual sign name.

### Implementation

Use the code cell (or multiple code cells, if necessary) to implement the first step of your project. Once you have completed your implementation and are satisfied with the results, be sure to thoroughly answer the questions that follow.

In [ ]:
### Load the images and plot them here.
### Feel free to use as many code cells as needed.

### Question 6

_Choose five candidate images of traffic signs and provide them in the report. Are there any particular qualities of the image(s) that might make classification difficult? It would be helpful to plot the images in the notebook._



**Answer:**

In [ ]:
### Run the predictions here.
### Feel free to use as many code cells as needed.

### Question 7

_Is your model able to perform equally well on captured pictures or a live camera stream when compared to testing on the dataset?_


**Answer:**

In [ ]:
### Visualize the softmax probabilities here.
### Feel free to use as many code cells as needed.
import cv2 as cv2
import matplotlib.pyplot as plt
import glob

%matplotlib inline

files = glob.glob('data/cIMG*.png')

my_images = np.ndarray(shape=(8,32,32), dtype=float)
for i, file in enumerate(files):
    im = cv2.imread(file)
    im = grayscale(im)
    im = cv2.equalizeHist(im, (0,254) )
    im = cv2.resize(im, (32,32))
    my_images[i, :, : ] = im

my_images.shape
plot_images(images=my_images, grayscale=True, n_cols=8)

In [ ]:
my_x = normalize_grayscale(flatten_all_gray(my_images))

sess = tf.Session()
new_saver = tf.train.import_meta_graph('models/lenet')
new_saver.restore(sess, tf.train.latest_checkpoint('./'))
all_vars = tf.trainable_variables()
for v in all_vars:
    print(v.name)

    
#validation_accuracy = session.run(accuracy, feed_dict=valid_feed_dict)
#y_pred = session.run(predicted_class, feed_dict=valid_feed_dict)
        

### Question 8

*Use the model's softmax probabilities to visualize the **certainty** of its predictions, [`tf.nn.top_k`](https://www.tensorflow.org/versions/r0.11/api_docs/python/nn.html#top_k) could prove helpful here. Which predictions is the model certain of? Uncertain? If the model was incorrect in its initial prediction, does the correct prediction appear in the top k? (k should be 5 at most)*


**Answer:**

### Question 9
_If necessary, provide documentation for how an interface was built for your model to load and classify newly-acquired images._


**Answer:**

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.